In [4]:
import networkx as nx
import pandas as pd
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import random
from itertools import islice
import time

In [5]:
# read demands
demand = open('Sprint/demand.txt', 'r')
lines = demand.readlines()
demands = []

for line in lines:
    demands.append(line.split())

demands = np.array(demands, dtype = float)
demands = np.max(demands, axis=0) #use the maximum demand over all times (rows) for each node-node pair

#demand = [float(i) for i in line.split()]
# n = int(len(demands)**0.5)
# demand = np.array(demands).reshape(n, n)

# np.fill_diagonal(demand, 0) #demand from a node to itself should be 0

In [6]:
def createGraph(n):
  edges = []
  cap = 1000000000
  for i in range(n-1):
      edges.append((int(i), int(i+1), int(cap)))
  edges.append((int(0), int(n-1), int(cap)))
  for i in range(n):
    for j in range(2):
      node = (i + random.randint(1, n-1)) % n
      tail = min(i, node)
      head = max(i, node)
      if (int(tail), int(head), int(cap)) not in edges:
          edges.append((int(tail), int(head), int(cap)))

  # generate graph
  G = nx.Graph()
  G.add_weighted_edges_from(edges)

  return G

In [7]:
def k_shortest_paths(G, source, target, k, weight=None):
    return list(
        islice(nx.shortest_simple_paths(G, source, target, weight=weight), k)
    )

In [8]:
def getDemand(n):
  newDemand = demands[np.random.choice(range(len(demands)), n**2)]
  newDemand = np.array(newDemand).reshape(n, n)
  np.fill_diagonal(newDemand, 0)
  return newDemand

In [9]:
def maxThroughput(G, newDemand, n):
    edges_undirected = []
    for u, v in G.edges():
        tail = min(u,v)
        head = max(u,v)
        edges_undirected.append((tail, head, G[u][v]['weight']))

    paths = []
    for i in range(n):
        for j in range(n):
            if i != j:
                newPaths = []
                for path in k_shortest_paths(G, i, j, 5):
                    newPaths.append([(path[i], path[i + 1]) for i in range(len(path) - 1)])
                # newPaths = list(sorted(nx.all_simple_edge_paths(G, i, j), key=len))[0:5]
                paths.extend(newPaths)

    # Create a new model
    m = gp.Model("Max Throughput")

    edges_vars = {} # maps edge to (edge var, [path vars that use it])
    for (src, dest, capacity) in edges_undirected:
        tail = min(src, dest)
        head = max(src, dest)
        edges_vars[(tail, head)] = (m.addVar(name=f'({tail}, {head})'), [])
        m.addConstr(edges_vars[(tail, head)][0] <= capacity, f'({tail}, {head})<={capacity}')   

    demand_vars = {} 

    for i in range(n):
        for j in range(n):
            demand_vars[(i, j)] = (newDemand[i][j],[])

    objective = 0 

    for path in paths:
        p = m.addVar(name=f'{path}')
        objective += p

        # add paths that try to satisfy a specific demand
        src, dest = path[0][0], path[-1][-1]
        demand_vars[(src, dest)][1].append(p)

        # this path uses edges --> need to 'credit' the edge
        for (node1, node2) in path:
            tail = min(node1, node2)
            head = max(node1, node2)
            edges_vars[(tail, head)][1].append(p)

    # relating edge to path variables
    for key in edges_vars:
        m.addConstr(edges_vars[key][0] == sum(edges_vars[key][1]))

    # do not send more than what is demanded
    for key in demand_vars:
        if demand_vars[key][1]:
            m.addConstr(sum(demand_vars[key][1]) <= demand_vars[key][0])

    # Set objective
    m.setObjective(objective, GRB.MAXIMIZE)

    # Optimize model
    m.optimize()

    # for v in m.getVars():
    #     print('%s %g' % (v.VarName, v.X))

    return m.Runtime

In [10]:
# Minimize the MLU (changing the objective function)
def minMLUObj(G, newDemand, n):
    flow = 0 # represents the total flow sent on this network

    edges_undirected = []
    for u, v in G.edges():
        tail = min(u,v)
        head = max(u,v)
        edges_undirected.append((tail, head, G[u][v]['weight']))

    paths = []
    for i in range(n):
        for j in range(n):
            if i != j:
                newPaths = []
                for path in k_shortest_paths(G, i, j, 5):
                    newPaths.append([(path[i], path[i + 1]) for i in range(len(path) - 1)])
                # newPaths = list(sorted(nx.all_simple_edge_paths(G, i, j), key=len))[0:5]
                paths.extend(newPaths)

    # Create a new model
    m = gp.Model("Min MLU - flow")

    # new variable for current MLU
    MLU = m.addVar(name="MLU")

    edges_vars = {} # maps edge to (edge var, [path vars that use it])
    for (src, dest, capacity) in edges_undirected:
        tail = min(src, dest)
        head = max(src, dest)
        edges_vars[(tail, head)] = (m.addVar(name=f'({tail}, {head})'), [])
        m.addConstr(edges_vars[(tail, head)][0] <= capacity, f'({src}, {tail})<={head}') # >= 0 is already added
        m.addConstr(edges_vars[(tail, head)][0] / capacity <= MLU)

    demand_vars = {} # maps demand between two nodes to the path variables that constitute it

    for i in range(n):
        for j in range(n):
            demand_vars[(i, j)] = (newDemand[i][j],[])


    for path in paths:
        p = m.addVar(name=f'{path}')
        flow += p # add this path flow to our total flow accumulator

        # add paths that try to satisfy a specific demand
        src, dest = path[0][0], path[-1][-1]
        demand_vars[(src, dest)][1].append(p)

        # this path uses edges --> need to 'credit' the edge
        for (node1, node2) in path:
            tail = min(node1, node2)
            head = max(node1, node2)
            edges_vars[(tail, head)][1].append(p)

    # relating edge to path variables
    for key in edges_vars:
        m.addConstr(edges_vars[key][0] == sum(edges_vars[key][1]))

    # do not send more than what is demanded
    for key in demand_vars:
        if demand_vars[key][1]:
            m.addConstr(sum(demand_vars[key][1]) <= demand_vars[key][0])

    # create a variable for the total flow in the network so we can retrieve its value after solving
    total_flow = m.addVar(name="total_flow")
    m.addConstr(total_flow == flow)

    # Set objective
    m.setObjective(MLU - total_flow/np.sum(demands), GRB.MINIMIZE) # minimize MLU but reward satisfying as much demand as possible

    # Optimize model
    m.optimize()

    # for v in m.getVars():
    #     print('%s %g' % (v.VarName, v.X))

    # print('Obj: %g' % m.ObjVal)
    return m.Runtime

In [11]:
# Minimize MLU (adding a constraint)
def minMLUConstraint(G, newDemand, n):
    flow = 0 # represents the total flow sent on this network

    edges_undirected = []
    for u, v in G.edges():
        tail = min(u,v)
        head = max(u,v)
        edges_undirected.append((tail, head, G[u][v]['weight']))

    paths = []
    for i in range(n):
        for j in range(n):
            if i != j:
                newPaths = []
                for path in k_shortest_paths(G, i, j, 5):
                    newPaths.append([(path[i], path[i + 1]) for i in range(len(path) - 1)])
                # newPaths = list(sorted(nx.all_simple_edge_paths(G, i, j), key=len))[0:5]
                paths.extend(newPaths)

    # Create a new model
    m = gp.Model("Min MLU with constraint")

    # new variable for MLU
    MLU = m.addVar(name="MLU")

    edges_vars = {} #maps edge to (edge var, [path vars that use it])
    for (src, dest, capacity) in edges_undirected:
        tail = min(src, dest)
        head = max(src, dest)
        edges_vars[(tail, head)] = (m.addVar(name=f'({tail}, {head})'), [])
        m.addConstr(edges_vars[(tail, head)][0] <= capacity, f'({src}, {tail})<={head}') # >= 0 is already added
        m.addConstr(edges_vars[(tail, head)][0] / capacity <= MLU)

    demand_vars = {} # maps demand between two nodes to the path variables that constitute it

    for i in range(n):
        for j in range(n):
            demand_vars[(i, j)] = (newDemand[i][j],[])

    for path in paths:
        p = m.addVar(name=f'{path}')
        flow += p # add this path flow to our total flow accumulator

        # add paths that try to satisfy a specific demand
        src, dest = path[0][0], path[-1][-1]
        demand_vars[(src, dest)][1].append(p)

        # this path uses edges --> need to 'credit' the edge
        for (node1, node2) in path:
            tail = min(node1, node2)
            head = max(node1, node2)
            edges_vars[(tail, head)][1].append(p)

    #relating edge to path variables
    for key in edges_vars:
        m.addConstr(edges_vars[key][0] == sum(edges_vars[key][1]))

    # do not send more than what is demanded
    for key in demand_vars:
        if demand_vars[key][1]:
            m.addConstr(sum(demand_vars[key][1]) >= demand_vars[key][0])

    # create a variable for the total flow in the network so we can retrieve its value after solving
    total_flow = m.addVar(name="total_flow")
    m.addConstr(total_flow == flow)

    # Set objective
    m.setObjective(MLU, GRB.MINIMIZE)

    # Optimize model
    m.optimize()

    # for v in m.getVars():
    #     print('%s %g' % (v.VarName, v.X))

    # print('Obj: %g' % m.ObjVal)
    return m.Runtime

In [12]:
res = {'maxThroughput' : [], 'minMLUObj' : [], 'minMLUConstraint': []}

In [13]:
G5 = createGraph(5)
demand5 = getDemand(5)
runtime5_maxThroughput = maxThroughput(G5, demand5, 5)
runtime5_minMLUObj = minMLUObj(G5, demand5, 5)
runtime5_minMLUConstraint = minMLUConstraint(G5, demand5, 5)

res['maxThroughput'].append(runtime5_maxThroughput)
res['minMLUObj'].append(runtime5_minMLUObj)
res['minMLUConstraint'].append(runtime5_minMLUConstraint)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 36 rows, 108 columns and 348 nonzeros
Model fingerprint: 0xb016a551
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+09]
Presolve removed 36 rows and 108 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1071998e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.107199790e+06
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|

In [14]:
G10 = createGraph(10)
demand10 = getDemand(10)
runtime10_maxThroughput = maxThroughput(G10, demand10, 10)
runtime10_minMLUObj = minMLUObj(G10, demand10, 10)
runtime10_minMLUConstraint = minMLUConstraint(G10, demand10, 10)

res['maxThroughput'].append(runtime10_maxThroughput)
res['minMLUObj'].append(runtime10_minMLUObj)
res['minMLUConstraint'].append(runtime10_minMLUConstraint)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 144 rows, 477 columns and 1504 nonzeros
Model fingerprint: 0x42a12e35
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 1e+09]
Presolve removed 144 rows and 477 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1669856e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.166985644e+07
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using u

In [15]:
G25 = createGraph(25)
demand25 = getDemand(25)
runtime25_maxThroughput = maxThroughput(G25, demand25, 25)
runtime25_minMLUObj = minMLUObj(G25, demand5, 25)
runtime25_minMLUConstraint = minMLUConstraint(G25, demand25, 25)

res['maxThroughput'].append(runtime25_maxThroughput)
res['minMLUObj'].append(runtime25_minMLUObj)
res['minMLUConstraint'].append(runtime25_minMLUConstraint)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [ ]:
G50 = createGraph(50)
demand50 = getDemand(50)
runtime50_maxThroughput = maxThroughput(G50, demand50, 50)
runtime50_minMLUObj = minMLUObj(G50, demand50, 50)
runtime50_minMLUConstraint = minMLUConstraint(G50, demand50, 50)

res['maxThroughput'].append(runtime50_maxThroughput)
res['minMLUObj'].append(runtime50_minMLUObj)
res['minMLUConstraint'].append(runtime50_minMLUConstraint)

In [ ]:
G100 = createGraph(100)
demand100 = getDemand(100)
runtime100_maxThroughput = maxThroughput(G100, demand100, 100)
runtime100_minMLUObj = minMLUObj(G100, demand100, 100)
runtime100_minMLUConstraint = minMLUConstraint(G100, demand100, 100)

res['maxThroughput'].append(runtime100_maxThroughput)
res['minMLUObj'].append(runtime100_minMLUObj)
res['minMLUConstraint'].append(runtime100_minMLUConstraint)

In [16]:
res

{'maxThroughput': [0.02500009536743164, 0.016000032424926758],
 'minMLUObj': [0.011999845504760742, 0.013000011444091797],
 'minMLUConstraint': [0.01900005340576172, 0.013000011444091797]}